# Grammar Scoring Competition

## 1. Approach Overview
The objective of this competition is to predict a grammar score, which is fundamentally a **regression problem**. The chosen approach involves conversion of provided audio files to transcripts using a pre-trained ASR model name Whisper. This was then fed as the fine-tuning training data to a transformer based model named DaBERT for grammar scoring task.

## 2. Preprocessing Steps
The initial preprocessing pipeline is designed to standardize the audio data before feature extraction:
* **Package Installation:** All required libraries (`torchaudio`, `wordfreq`, etc.) are installed.
* **Audio Standardization:** The core `load_and_resample` function ensures all raw audio files are loaded and resampled to a consistent rate of **16,000 Hz**.
* **Mono Conversion:** Multi-channel audio (e.g., stereo) is converted to **mono** (single channel) by averaging the channels, which is standard practice for speech processing.


## 3. Pipeline Architecture
The machine learning pipeline follows a standard supervised learning flow:
1.  **Raw Input:** Audio File + Ground Truth Score.
2.  **Audio Preprocessing:** Resampling (16kHz) and Mono Conversion.
3.  **Speech to text:** The audio was then converted to raw text using ASR model name **Whisper**. The generated texts were saved inform of csv.
4.  **Model Training:** Training the **transformer** on the saved text csv.
5.  **Evaluation:** Performance is assessed using **Root Mean Square Error (RMSE)**.
## 4. Evaluation Results
The final results from the model run are presented below.

| Transformer model | Train RMSE | Leaderboard Score
| :--- | :--- | :--- |
| DaBERT |  0.16188851 | 0.600
| DaBERT-small|  0.16698851 | 0.599
| DaBERT-large |  0.1449279 | 0.735

In [ ]:
!pip install wordfreq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.1/183.1 kB 14.8 MB/s eta 0:00:00


In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.4 MB/s eta 0:00:00


In [ ]:
!pip install TorchCodec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 82.6 MB/s eta 0:00:00


In [ ]:
!pip install transformers==4.57.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 124.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.2
    Uninstalling transformers-4.57.2:
      Successfully uninstalled transformers-4.57.2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Visualizing waveform

In [ ]:
import matplotlib.pyplot as plt
import torch
import torchaudio
import torchaudio.transforms as T
import torchaudio.functional as F

In [ ]:
# Function to handle the actual plotting logic for waveform or spectrogram.
def _plot(waveform, sample_rate, title):
  """
  Internal helper function to plot the waveform or spectrogram.
  Handles conversion to numpy and setting up the plot axes.
  """
  # Convert PyTorch tensor to NumPy array for plotting
  waveform = waveform.numpy()

  num_channels, num_frames = waveform.shape
  # Create a time axis based on the number of frames and sample rate
  time_axis = torch.arange(0, num_frames) / sample_rate

  figure, axes = plt.subplots(num_channels, 1)
  if num_channels == 1:
    axes = [axes]
  for c in range(num_channels):
    # Plot the waveform vs time for the Waveform visualization
    if title == "Waveform":
      axes[c].plot(time_axis, waveform[c], linewidth=1)
      axes[c].grid(True)
    # Plot the spectrogram (frequency vs time)
    else:
      axes[c].specgram(waveform[c], Fs=sample_rate)
    if num_channels > 1:
      axes[c].set_ylabel(f'Channel {c+1}')
  figure.suptitle(title)
  plt.show(block=False)

# Public function to display the audio waveform (amplitude over time).
def plot_waveform(waveform, sample_rate):
  """Plots the time-domain waveform of the audio signal."""
  _plot(waveform, sample_rate, title="Waveform")

# Public function to display the audio spectrogram (frequency content over time).
def plot_specgram(waveform, sample_rate):
  """Plots the spectrogram of the audio signal (currently not used but included for completeness)."""
  _plot(waveform, sample_rate, title="Spectrogram")

In [ ]:
# Purpose: Ensure all audio files have the same format:
# ✔ Same sample rate (e.g., 16kHz)
# ✔ Converted to mono (1 channel)

def load_and_resample(path, target_sr=16000):
    # Load the audio file from the specified path, obtaining the waveform tensor and original sample rate (sr).
    waveform, sr = torchaudio.load(path)  # shape: [channels, time]

    # Check if the original sample rate (sr) matches the target rate.
    if sr != target_sr:
        # Initialize the Resample transform from torchaudio.
        resampler = T.Resample(orig_freq=sr, new_freq=target_sr)
        # Apply the resampling transformation to the waveform.
        waveform = resampler(waveform)

    # Convert to mono if the audio has multiple channels (e.g., stereo)
    if waveform.shape[0] > 1:
        # Average the channels along the first dimension to create a single mono channel
        waveform = torch.mean(waveform, dim=0, keepdim=True)

    return waveform, target_sr

In [ ]:
file_path = "drive/MyDrive/grammar_scoring/audios/train/audio_1.wav"
waveform, sample = load_and_resample(file_path)
plot_waveform(waveform, sample)
waveform = waveform.squeeze() #always squeeze waveform to avoid dimension related errors

## Speech to Text

### Whisper
Loading and tesing Whisper model

In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration, pipeline

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
# # load model and processor

#The Whisper model is intrinsically designed to work on audio samples of up to 30s in duration.
#However, by using a chunking algorithm, it can be used to transcribe audio samples of up to arbitrary length.
#This is possible through Transformers pipeline method. Chunking is enabled by setting chunk_length_s=30 when instantiating the pipeline.

pipe = pipeline(
  "automatic-speech-recognition",
  model="openai/whisper-medium",
  chunk_length_s=30,
  stride_length_s=2,
  device=device,
)

In [ ]:
#this is the main function where transcripts are generated from audio
def transcript(file_name):
  file_path = "drive/MyDrive/grammar_scoring/audios/train/" + file_name
  waveform, sample = load_and_resample(file_path)
  waveform = waveform.squeeze()
  # input_features = processor(waveform, sampling_rate=sample, return_tensors="pt").input_features
  # predicted_ids = model.generate(input_features)
  # transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
  transcription = pipe(waveform, batch_size=8, return_timestamps=True)["text"]
  # print(transcription)
  return transcription


In [ ]:
# #testing with an audio file
file_path = "audio_1.wav"
print(transcript(file_path))

### Generating csv
After loading and testing the pre-trained Whisper model, the audios from train dataset were now converted to their respective transcript which was saved in the form of csv.

In [ ]:
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#creating dataframe
df = pd.DataFrame(columns=["filename", "transcript"])

In [ ]:
dir_path = "drive/MyDrive/grammar_scoring/audios/train"
for files in os.listdir(dir_path):

  name = os.path.splitext(os.path.basename(files))[0]
  print(name)

  # speech to audio convertion
  trans = transcript(files)
  print(trans)

  df.loc[len(df)] = {'filename': name, 'transcript': trans}


In [ ]:
df.to_csv("drive/MyDrive/grammar_scoring/csvs/transcript_train.csv", index=False)

In [ ]:
df

###data cleaning

In [ ]:
import pandas as pd

In [ ]:
df_main = pd.read_csv("drive/MyDrive/grammar_scoring/csvs/train.csv")
df_train = pd.read_csv("drive/MyDrive/grammar_scoring/csvs/transcript_train.csv")

In [ ]:
df_final = pd.merge(df_train, df_main, on="filename", how="inner")

In [ ]:
df_final

,filename,transcript,label
0,audio_106,The best day of my life would definitely be w...,5.0
1,audio_100,The moment in my life that I cherish the most...,3.0
2,audio_102,My favorite place to visit has always been re...,3.0
3,audio_10,"Okay, so my role model is someone like my mot...",3.0
4,audio_107,This goal is important to me because initiall...,3.5
...,...,...,...
425,audio_81,Hello this is Aisona. Today I will describe a...,2.5
426,audio_89,"My school playground, it was huge. It was ver...",2.0
427,audio_97,I can hear many cars driving by and lot of pe...,2.0
428,audio_94,The playground looks like a grain filled with...,3.5


In [ ]:
# removing rows having non-english characters
# Regex pattern allowing only English letters, digits, whitespace, and some punctuation
import re
pattern = re.compile(r'^[\x00-\x7F]*$')

# Function to test each cell (convert to string to avoid errors)
def is_clean(value):
    return bool(pattern.match(str(value)))

# Keep rows where **all columns** satisfy the condition
clean_df = df_final[df_final.apply(lambda row: all(is_clean(x) for x in row), axis=1)]
clean_df

,filename,transcript,label
0,audio_106,The best day of my life would definitely be w...,5.0
1,audio_100,The moment in my life that I cherish the most...,3.0
2,audio_102,My favorite place to visit has always been re...,3.0
3,audio_10,"Okay, so my role model is someone like my mot...",3.0
4,audio_107,This goal is important to me because initiall...,3.5
...,...,...,...
425,audio_81,Hello this is Aisona. Today I will describe a...,2.5
426,audio_89,"My school playground, it was huge. It was ver...",2.0
427,audio_97,I can hear many cars driving by and lot of pe...,2.0
428,audio_94,The playground looks like a grain filled with...,3.5


In [ ]:
clean_df = df_final.drop(columns=["filename"])
clean_df = clean_df.rename(columns={"label": "labels"})

## Transformer model for regression
A pre-trained DaBERT model was fine-tuned as a regressor for grammar scoring task. General steps like loading and tokenizing were done as per HuggingFace documentation.

In [ ]:
import pandas as pd
from datasets import Dataset, Value
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
import torch
import evaluate

In [ ]:
dataset_train = Dataset.from_pandas(clean_df)
dataset_train = dataset_train.cast_column("labels", Value("float32"))

Casting the dataset:   0%|          | 0/430 [00:00<?, ? examples/s]

In [ ]:
model_name = "microsoft/deberta-v3-large"   # recommended for grammar scoring
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/580 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
def tokenize_fn(batch):
    return tokenizer(
        batch["transcript"],
        padding="max_length",
        truncation=True,
        max_length=128,
    )

train_ds = dataset_train.map(tokenize_fn, batched=True)

Map:   0%|          | 0/430 [00:00<?, ? examples/s]

In [ ]:
train_ds = train_ds.remove_columns(
    [col for col in train_ds.column_names if col not in ["input_ids","attention_mask","labels"]]
)

train_ds.set_format(type="torch")

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=1,                 # regression
    problem_type="regression"
)

pytorch_model.bin:   0%|          | 0.00/874M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
mse = evaluate.load("mse")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.squeeze()
    return {"mse": mse.compute(predictions=preds, references=labels)["mse"]}

model.safetensors:   0%|          | 0.00/874M [00:00<?, ?B/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="./grammar_model",
    num_train_epochs=40,
    per_device_train_batch_size=16,
    learning_rate=2e-5,
    logging_steps=50,
    save_steps=500,          # optional
    load_best_model_at_end=False,   # important: no eval → cannot pick “best”
)


In [ ]:
class RMSETrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        # Extract labels
        labels = inputs.pop("labels")

        # Forward pass
        outputs = model(**inputs)
        logits = outputs.logits.squeeze()

        # MSE loss
        mse = torch.nn.functional.mse_loss(logits, labels)

        # RMSE = sqrt(MSE)
        rmse = torch.sqrt(mse)

        return (rmse, outputs) if return_outputs else rmse

In [ ]:
trainer = RMSETrainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    tokenizer=tokenizer
)

/tmp/ipython-input-4251641263.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `RMSETrainer.__init__`. Use `processing_class` instead.
  trainer = RMSETrainer(


In [ ]:
#epoch = 100
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 2, 'bos_token_id': 1}.
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: muskaan-maurya06 (muskaan-maurya06-none) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
50,1.045700
100,0.644900
150,0.466300
200,0.377000
250,0.296800
300,0.280700
350,0.253000
400,0.215000
450,0.207800
500,0.175200


TrainOutput(global_step=1080, training_loss=0.2524117405767794, metrics={'train_runtime': 845.8361, 'train_samples_per_second': 20.335, 'train_steps_per_second': 1.277, 'total_flos': 4007318375731200.0, 'train_loss': 0.2524117405767794, 'epoch': 40.0})

In [ ]:
trainer.save_model("drive/MyDrive/grammar_scoring/grammar_model_dabert_large4")

## Full model pipeline for inferencing

In [ ]:
#step 1: Whisper for audio to text conversion
import torch
import torchaudio
import torchaudio.transforms as T
import torchaudio.functional as F
from transformers import WhisperProcessor, WhisperForConditionalGeneration, pipeline

device = "cuda:0" if torch.cuda.is_available() else "cpu"

#waveform load function
def load_and_resample(path, target_sr=16000):
    waveform, sr = torchaudio.load(path)  # shape: [channels, time]
    if sr != target_sr:
        resampler = T.Resample(orig_freq=sr, new_freq=target_sr)
        waveform = resampler(waveform)
    # convert to mono (average channels)
    if waveform.shape[0] > 1:
        waveform = torch.mean(waveform, dim=0, keepdim=True)
    return waveform, target_sr

#Whisper model defintion
pipe = pipeline(
  "automatic-speech-recognition",
  model="openai/whisper-medium",
  chunk_length_s=30,
  stride_length_s=2,
  device=device
)

#audio to text function
def transcript(file_name):
  file_path = "drive/MyDrive/grammar_scoring/audios/test/" + file_name
  waveform, sample = load_and_resample(file_path)
  waveform = waveform.squeeze()
  # input_features = processor(waveform, sampling_rate=sample, return_tensors="pt").input_features
  # predicted_ids = model.generate(input_features)
  # transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
  transcription = pipe(waveform, batch_size=8, return_timestamps=True)["text"]
  # print(transcription)
  return transcription


Device set to use cuda:0
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


In [ ]:
# fine-tuned transformer(dabert) for text preprocessing and grammar scoring
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)

model_name = "/content/drive/MyDrive/grammar_scoring/grammar_model_dabert_large3"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# def score_batch(sentences):
#     inputs = tokenizer(
#         sentences,
#         return_tensors="pt",
#         padding=True,
#         truncation=True,
#         max_length=128,
#     )

#     with torch.no_grad():
#         outputs = model(**inputs)

#     scores = outputs.logits.squeeze().tolist()
#     return scores

def score_text(sentence):
    inputs = tokenizer(
        sentence,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=128,
        device=device,
    )

    with torch.no_grad():
        outputs = model(**inputs)

    score = outputs.logits.squeeze().item()
    return score


In [ ]:
import os
import pandas as pd

file_path = "drive/MyDrive/grammar_scoring/audios/test"
test_df =  pd.DataFrame(columns=["filename", "label"])

for files in os.listdir(file_path):
  name = os.path.splitext(os.path.basename(files))[0]
  print(name)
  # speech to audio convertion
  trans = transcript(files)
  print(trans)
  score = score_text(trans)
  print(score)

  test_df.loc[len(test_df)] = {'filename': name, 'label': score}



audio_106


Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.


 The best day of my life would definitely be when I went to the museum with my family. For me, it was one of the best days because it was such a nice time. It was my first time at the museum and my breath was taken away by the architecture, the art itself, and where it's located. It's such a beautiful location and it brought me so much peace. It really helped me. It helped me throughout the days because I felt the happiness because of me being there and with the people I most love and getting matcha. And the air was so crisp and beautiful. The weather was spectacular. There was nothing wrong with that day. And then we went to go eat dinner. a really nice restaurant so even though it's a very simple day but I really loved it.
5.195842266082764
audio_102_1
 My favorite place to visit has always been renting a cabin in southern Indiana or Kentucky. The reason that it's my favorite is because it's very secluded, surrounded by woods. There's a lot of nature around you. It's usually about a 

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


3.630868434906006
audio_1
 There could be a lot of different moments that I can say would be my favorite with my family. First off, it's very simple, like going out, eating out at a restaurant, and as long as the family is complete and together, then that is something that I would be really happy about. Another, another example would be when we go to the mall, for example, or any place which is very cold because, you know, here in the Philippines it's very hot, so I wanted to spend time with my family somewhere cold, like a mall.
3.099558115005493
audio_101
 My hobby is DIY or basically do it yourself. What I enjoy most about my hobby is making something out of raw materials. And of course, to do this, you would have to use a various amount of tools. The mediums I love to work with is metal, wood, or just building computers, building bikes, building things out of wood, basically just DIY, finding the little kinks and how things work and then building upon it. This hobby is done by myse

...

 మాలిసిక మాలిసిక మాలిసిక మాలిసిక మాలిసిక మాలిసిక మాలిసిక మాలిసిక మాలిసిక మాలిసిక మాలిసిక మాలిసిక మాలిసిక మాలిసిక మాలిసిక మాలిసిక మాలిసిక మాలిసిక మాలిసిక మాలిసిక మాలిసిక మాలిసిక మాలిసిక మాలిసిక మాలిసిక మాలిసిక మాలిసిక మాలిసిక మాలిసిక మాల్ల్ల్ల్ల్ల్ల్ల్లో మాల్ల్ల్ల్ల్లో మాల్ల్ల్ల్లో మాల్ల్ల్ల్లో మాల్ల్ల్ల్లో మాల్ల్ల్లో మాల్ల్ల్లో మాల్ల్ల్లో మాల్ల్ల్లో మాల్ల్ల్లో మాల్ల్ల్లో మాల్ల్ల్లో మాల్ల్ల్లో మాల్ల్ల్లో మాల్ల్ల్లో మాల్ల్ల్లో మాల్ల్ల్లో మాల్ల్ల్లో మాల్ల్లో మాల్ நான் இங்கே பார்க்கலாம் என்று நான் பார்க்கலாம் என்று நான் பார்க்கலாம் என்று நான் பார்க்கலாம் என்று நான் பார்க்கலாம் என்று நான் பார்க்கலாம் என்று நான் பார்க்கலாம் என்று நான் பார்க்கலாம் என்று நான் பார்க்கலாம் என்று நான் பார்க்கலாம் என்று நான் பார்க்கலாம் என்று நான் பார்க்கலாம் என்று நான் பார்க்கலாம் என்று நான் பார்க்கலாம் என்று நான் பார்க்கலாம் என்று நான் பார்க்கலாம் என்று நான் பார்க்கலாம் என்று நான் பார்க்கலாம் என்று நான் பார்க்கலாம் என்று நான் பார்க்கலாம் என்று நான் பார்க்கலாம் என்று நான் பார்க்கலாம் என்று நான் பார்க்கலாம் என்று 

Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


 When you go to the airport, the staff is very friendly and when you go to the inside of the airport, there are many foreigners and family are sad because their loved ones are leaving out of the country.
2.4749131202697754
audio_70


Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


 My favorite place is in Andhra Pradesh. It is in Chittoor district. It is the temple of Lord Venkateshwara Swamy. It has 7 hills to visit the temple. It is a good place to visit. ముందిలు ముందిలు ముందిలు ముందిలు ముందిలు ముందిలు ముందిలు ముందిలు ముందిలు ముందిలు ముందిలు ముందిలు ముందిలు ముందిలు ముందిలు ముందిలు ముందిలు ముందిలు ముందిలు ముందిలు ముందిలు ముందిలు ముందిలు ముందిలు ముందిలు ముందిలు ముందిలు ముందిలు ముందిలు ముందిరిమానా లాడి ఇదిరిమానా ఇదిరిమానా ఇదిరిమానా ఇదిరిమానా ఇదిరిమానా ఇదిరిమానా ఇదిరిమానా ఇదిరిమానా ఇదిరిమానా ఇదిరిమానా ఇదిరిమానా ఇదిరిమానా ఇదిరిమానా ఇదిరిమానా ఇదిరిమానా ఇదిరిమానా ఇదిరిమానా ఇదిరిమానా ఇదిరిమానా ఇదిరిమానా ఇదిరిమానా ఇదిరిమా
2.5732250213623047
audio_74
 A crowded market is easily spotted when you have a lot of small establishments where they sell a lot of things, different things, and they will have food, they will have clothes for sale, they will have a lot of games and cell phones to sell, as well as they will have a lot of people going there to purchase things for chea

In [ ]:
test_df.to_csv("output4.csv", index=False)